In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import re

# Preparation

In [3]:
driver = webdriver.Chrome()

driver.get("https://map.naver.com/")

In [18]:
css_selector = ".input_search"
elem = driver.find_element(by="css selector", value=css_selector)

elem.send_keys("신사역 맛집")
elem.send_keys(Keys.RETURN)

In [64]:
driver.switch_to.frame("searchIframe")

In [ ]:
for i in range(1, 11):
    css_selector = f"/html/body/div[3]/div/div[3]/div[1]/ul/li[{i}]/div[1]/a"
    elem = driver.find_element(By.XPATH, value=css_selector )

    elem.click()

In [34]:
# css_selector = '//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/a'
css_selector = "/html/body/div[3]/div/div[3]/div[1]/ul/li[1]/div[1]/a"
elem = driver.find_element(By.XPATH, value=css_selector )

elem.click()

In [65]:
css_selector = "/html/body/div[3]/div/div[3]/div[1]/ul/li[15]/div[1]/a"
elem = driver.find_element(By.XPATH, value=css_selector )

elem.click()

In [66]:
driver.switch_to.default_content()
driver.switch_to.frame("entryIframe")

In [67]:
# 인스타그램 계정 여부 확인 및 링크 get
try:
    elem = driver.find_element(By.XPATH, value= "//a[contains(text(), '인스타그램')]")
    href = elem.get_attribute('href')

    print(f"인스타그램 계정: {href}")
except:
    print("인스타그램 계정 없음")
    # continue

인스타그램 계정: https://www.instagram.com/hanol_garosu


In [68]:
elem = driver.find_element(By.XPATH, value="/html/body/div[3]/div/div/div/div[2]/div[1]/div[1]/div/span[1]")
store_name = elem.text

elem = driver.find_element(By.XPATH, value="/html/body/div[3]/div/div/div/div[2]/div[1]/div[1]/div/span[2]")
store_category = elem.text

print(f"매장 이름: {store_name}")
print(f"매장 분류: {store_category}")

매장 이름: 한올
매장 분류: 양식


In [71]:
try:
    elem = driver.find_element(By.XPATH, value="//a[contains(text(), '방문자리뷰')]")
    text = elem.text
    text = text.replace(",", "")

    visit_review_num = re.findall(r'\d+', text)[0]
    if visit_review_num:
        print(f"방문자 리뷰 {visit_review_num}건")
except:
    print("방문자 리뷰 없음")

방문자 리뷰 288건


In [72]:
try:
    elem = driver.find_element(By.XPATH, value="//a[contains(text(), '블로그리뷰')]")
    text = elem.text
    text = text.replace(",", "")

    blog_review_number = re.findall(r'\d+', text)[0]
    if blog_review_number:
        print(f"블로그 리뷰 {blog_review_number}건")
except:
    print("블로그 리뷰 없음")

블로그 리뷰 1144건


In [73]:
try:
    elem = driver.find_element(By.XPATH, value="/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/div")
    text = elem.text

    numbers = re.findall(r'\d+', text)
    if numbers:
        distance_from_station_exit = numbers[-1]
        print(f"지하철역으로부터의 거리: {distance_from_station_exit}m")

except NoSuchElementException:
    print("해당 XPath를 가진 div 태그가 없습니다.")

지하철역으로부터의 거리: 421m


In [74]:
driver.switch_to.default_content()

# Merge

## ver.1


In [17]:
driver = webdriver.Chrome()

driver.get("https://map.naver.com/")

wait = WebDriverWait(driver, 10)

css_selector = ".input_search"
elem = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector)))

# 검색어 입력
elem.send_keys("신사역 맛집")
elem.send_keys(Keys.RETURN)


p = 0
while True:
    for i in range(10,15):
    # for i in range(1,11):

        
        print(f"========={p*54 + i}번째 매장=========")

        # 가장 상위 frame으로 이동 후 searchIframe으로 이동
        driver.switch_to.default_content()
        wait = WebDriverWait(driver, 10)
        wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))

        # i번째 매장에 대한 <a> 태그 찾아서 해상 위치까지 이동 후 click
        xpath = f"""//*[@id="_pcmap_list_scroll_container"]/ul/li[{i}]/div[1]/a"""
        
        time.sleep(1)

        wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
        elem = driver.find_element(By.XPATH, xpath)

        # 스크롤을 해당 요소 위치까지 이동 후 클릭
        if i!=1:
            driver.execute_script("arguments[0].scrollIntoView(true);", elem)
        elem.click()

        time.sleep(3)

        # 가장 상위 frame으로 이동 후 entryIframe으로 이동
        driver.switch_to.default_content()
        wait = WebDriverWait(driver, 10)
        wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))
        # entry_iframe = driver.find_element(By.ID, "entryIframe")
        # driver.switch_to.frame(entry_iframe)

        time.sleep(1)

        # 매장 이름 get
        store_name_xpath = """//*[@id="_title"]/div/span[1]"""
        elem = wait.until(EC.visibility_of_element_located((By.XPATH, store_name_xpath)))
        store_name = elem.text

        # 매장 분류 get
        store_category_xpath = """//*[@id="_title"]/div/span[2]"""
        elem = wait.until(EC.visibility_of_element_located((By.XPATH, store_category_xpath)))
        store_category = elem.text

        print(f"매장 이름: {store_name}")
        print(f"매장 분류: {store_category}")
        
        # 인스타그램 계정 여부 확인 및 링크 get
        try:
            elem = driver.find_element(By.XPATH, value= "//a[contains(text(), '인스타그램')]")
            href = elem.get_attribute('href')

            print(f"인스타그램 계정: {href}")
        except:
            print("인스타그램 계정 없음")
            continue

        # 방문자 리뷰 개수
        try:
            elem = driver.find_element(By.XPATH, value="//a[contains(text(), '방문자리뷰')]")
            text = elem.text
            text = text.replace(",", "")

            visit_review_num = re.findall(r'\d+', text)[0]
            if visit_review_num:
                print(f"방문자 리뷰 {visit_review_num}건")
        except:
            print("방문자 리뷰 없음")

        # 블로그 리뷰 개수
        try:
            elem = driver.find_element(By.XPATH, value="//a[contains(text(), '블로그리뷰')]")
            text = elem.text
            text = text.replace(",", "")

            blog_review_number = re.findall(r'\d+', text)[0]
            if blog_review_number:
                print(f"블로그 리뷰 {blog_review_number}건")
        except:
            print("블로그 리뷰 없음")

        # 지하철역 출구로부터 거리
        try:
            elem = wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/div")))
            text = elem.text

            # station = re.findall(r'\b\w+역\b',text)
            numbers = re.findall(r'\d+', text)
            # if station:
            #     print(f"지하철역: {station[0]}")
            if numbers:
                distance_from_station_exit = numbers[-1]
                print(f"지하철역으로부터의 거리: {distance_from_station_exit}m")
        except NoSuchElementException:
            print("해당 XPath를 가진 div 태그가 없습니다.")

        try:
            elements = driver.find_elements(By.XPATH, "//span[text()='TV방송정보']")
            if len(elements) > 0:
                print("방송출연 여부: O")
            else:
                print("방송출연 여부: X")
        except NoSuchElementException:
            print("방송출연 여부: X")

        # 정보 탭으로 이동
        try:
            # elem = driver.find_element(By.XPATH, "//span[contains(text(),'리뷰')]")
            # actions = ActionChains(driver)
            # actions.move_to_element(elem).perform()

            # time.sleep(3)

            # xpath = "//a[.//span[contains(text(),'다음')]]"
            # xpath = """//div[contains(@class, 'flicking-viewport')]//a[.//span[contains(text(),'다음')]]"""

            css_selector = "/html/body/div[3]/div/div/div/div[4]/div/div/div/a[2]/span"

            elem = driver.find_elements(By.CSS_SELECTOR, css_selector)
            print(elem)
            print(elem.tag_name)
            actions = ActionChains(driver)
            actions.move_to_element(elem).perform()

            time.sleep(5)

            actions.click(elem).perform()
            # elem = driver.find_element(By.XPATH, """//*[@id="app-root"]/div/div/div/div[4]/div/div/div/a[2]""")
            # elem.click()

            # xpath = "//a[.//span[contains(text(),'다음')]]"
            # elem = driver.find_element(By.XPATH, xpath)
            # ActionChains(driver).move_to_element(elem).click(elem).perform()
        except ElementNotInteractableException:
            print("skip")
            pass
        except:
            pass

        try:
            xpath = "//a[.//span[contains(text(),'정보')]]"
            elem = driver.find_element(By.XPATH, xpath)
            elem.click()

            time.sleep(0.5)

            # 주차 가능 여부
            try:
                free_parking_xpath = """//*[@id="app-root"]/div/div/div/div[6]/div/div[4]/div/div/div/div/div[2]/div/div"""
                free_parking_elem = driver.find_element(By.XPATH, free_parking_xpath)
                if free_parking_elem != None:
                    print("무료 주차 여부: O")
            except NoSuchElementException:
                print("무료 주차 여부: X")

                try:
                    paid_parking_xpath = """//*[@id="app-root"]/div/div/div/div[6]/div/div[3]/div/div/div[1]/div[1]/div[2]/div[1]/div"""
                    paid_parking_elem = driver.find_element(By.XPATH, paid_parking_xpath)
                    if paid_parking_elem != None:
                        print("유료 주차 여부: O")
                except NoSuchElementException:
                    print("유료 주차 여부: X")

        except NoSuchElementException:
            print("주차 가능 여부 알 수 없음")

        
    
    driver.switch_to.default_content()
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))
    next_page_button = driver.find_element(By.XPATH, "//a[span[contains(text(),'다음페이지')]]")
    next_page_button.click()
    p += 1

=========10번째 매장=========


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.129)
Stacktrace:
0   chromedriver                        0x00000001031444dc chromedriver + 4162780
1   chromedriver                        0x000000010313c664 chromedriver + 4130404
2   chromedriver                        0x0000000102d93bc0 chromedriver + 293824
3   chromedriver                        0x0000000102d6c6c0 chromedriver + 132800
4   chromedriver                        0x0000000102e0525c chromedriver + 758364
5   chromedriver                        0x0000000102e19294 chromedriver + 840340
6   chromedriver                        0x0000000102dcd6bc chromedriver + 530108
7   chromedriver                        0x0000000102dce930 chromedriver + 534832
8   chromedriver                        0x0000000103109e08 chromedriver + 3923464
9   chromedriver                        0x000000010310e3dc chromedriver + 3941340
10  chromedriver  

## ver.2

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

import time
import re

In [3]:
empty_searchIframe = """//*[@id="_pcmap_list_scroll_container"]"""
empty_entryIframe = """//*[@id="app-root"]"""
empty_root = """//*[@id="root"]"""

df = pd.DataFrame(columns=['name', 'category', 'instagram_link', 'visitor_review_count', 
                           'blog_review_count', 'distance_from_subway', 'on_tv', 'is_parking', 
                           'free_parking', 'valet_parking', 'no_kids', 'no_pet', 'seoul_michelin'])

In [14]:
driver = webdriver.Chrome()
driver.get("https://map.naver.com/")

wait = WebDriverWait(driver, 10)
driver.implicitly_wait(10)
driver.find_element(By.XPATH, empty_root)

# 검색 Field
css_selector = ".input_search"
elem = driver.find_element(By.CSS_SELECTOR, css_selector)
time.sleep(1.5) # 페이지가 완전히 로딩되기 위한 시간 고려
elem.send_keys("신사역 맛집")
time.sleep(1.5) # 검색엔진 작동을 위한 시간 고려
elem.send_keys(Keys.RETURN)

count = 0
while True:
    for i in range(1,51):
        count += 1

        store_dict = {
            'name': "",
            'category': "",
            "instagram_link": None,
            "visitor_review_count": 0,
            "blog_review_count": 0,
            "distance_from_subway": 0,
            "on_tv": False,
            "is_parking": False,
            "free_parking": False,
            "valet_parking":False,
            "no_kids": False,
            "no_pet":True,
            "seoul_michelin":False,
        }

        print(f"==============={count}번째 매장===============")

        # Search Iframe
        driver.switch_to.default_content()
        driver.find_element(By.XPATH, empty_root)        
        searchIframe = driver.find_element(By.XPATH, """//*[@id="searchIframe"]""")
        driver.switch_to.frame(searchIframe)
        driver.find_element(By.XPATH, empty_searchIframe)
        
        # 매장 클릭
        xpath = f"""//*[@id="_pcmap_list_scroll_container"]/ul/li[{i}]/div[1]/a[1]"""
        elem = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
        time.sleep(2)
        if i!=1:
            driver.execute_script("arguments[0].scrollIntoView(true);", elem)
        driver.execute_script("arguments[0].click();", elem)

        # Entry Iframe
        driver.switch_to.default_content()
        driver.find_element(By.XPATH, empty_root)
        wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))
        driver.find_element(By.XPATH, empty_entryIframe)

        # 매장 이름, 카테고리
        store_name_xpath = """//*[@id="_title"]/div/span"""
        elem = wait.until(EC.visibility_of_all_elements_located((By.XPATH, store_name_xpath)))
        store_dict['name'] = elem[0].text
        store_dict['category'] = elem[1].text
        
        # 인스타그램
        # 인스타그램 계정 없을 경우, DataFrame에 저장하지 않고 다음 매장으로 skip
        try:
            elem = driver.find_element(By.XPATH, value= "//a[contains(text(), '인스타그램')]")
            store_dict['instagram_link'] = elem.get_attribute('href')
        except NoSuchElementException:
            print("인스타그램 없음")
            continue

        # 방문자 리뷰, 블로그 리뷰 개수
        try:
            elem_visitor = driver.find_element(By.XPATH, value="//a[contains(text(), '방문자리뷰')]")
            elem_blog = driver.find_element(By.XPATH, value="//a[contains(text(), '블로그리뷰')]")

            visitor_review_count = int(re.findall(r'\d+', elem_visitor.text.replace(",", ""))[0])
            blog_review_count = int(re.findall(r'\d+', elem_blog.text.replace(",", ""))[0])

            store_dict['visitor_review_count'] = visitor_review_count
            store_dict['blog_review_count'] = blog_review_count

        except NoSuchElementException:
            store_dict['visitor_review_count'] = None
            store_dict['blog_review_count'] = None

        # # 지하철역 출구로부터 거리
        # try:
        #     # outer_div_xpath = """//div[.//strong[.//svg[.//path[@d='M9 1C5.4 1 2.5 3.7 2.5 7.1c0 1.2.4 2.3 1 3.3l5.1 6.1c.2.2.6.2.8 0l5.1-6.1c.7-1 1-2.1 1-3.3C15.5 3.7 12.6 1 9 1zm0 8c-.8 0-1.5-.7-1.5-1.5S8.2 6 9 6s1.5.7 1.5 1.5S9.8 9 9 9z']]]]"""
        #     # """M9 1C5.4 1 2.5 3.7 2.5 7.1c0 1.2.4 2.3 1 3.3l5.1 6.1c.2.2.6.2.8 0l5.1-6.1c.7-1 1-2.1 1-3.3C15.5 3.7 12.6 1 9 1zm0 8c-.8 0-1.5-.7-1.5-1.5S8.2 6 9 6s1.5.7 1.5 1.5S9.8 9 9 9z"""
        #     # outer_div_elem = driver.find_element(By.XPATH, outer_div_xpath)
        #     # print(1)
        #     # inner_div_elem = outer_div_elem.find_element(By.XPATH, ".//div/div")
        #     # print(2)
        #     # inner_html = inner_div_elem.get_attribute('innerHTML')
        #     # print(3)
        #     # inner_text = re.search(r'">(.*?)<span>', inner_html)
        #     # print(4)

        #     # print(inner_text)
        #     print("역역역역역역역역역역")
        #     div_elem = wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/div")))
        #     print(2)
        #     inner_html = div_elem.get_attribute('innerHTML')
        #     print(inner_html)
            
        #     text_without_tags = re.sub(r'<[^>]+>', '', inner_html)
        #     cleaned_text = re.sub(r'<!--.*?-->', '', text_without_tags)

        #     # 결과 출력
        #     print(cleaned_text.strip())

        #     # # station = re.findall(r'\b\w+역\b',text)
        #     # numbers = re.findall(r'\d+', text)
        #     # # if station:
        #     # #     print(f"지하철역: {station[0]}")
        #     # if numbers:
        #     #     distance_from_station_exit = numbers[-1]
        #     #     print(f"지하철역으로부터의 거리: {distance_from_station_exit}m")
        # except NoSuchElementException:
        #     print("해당 XPath를 가진 div 태그가 없습니다.")

        # 방송 출연
        try:
            element = driver.find_element(By.XPATH, "//strong[descendant::span[text()='TV방송정보']]")
            store_dict['on_tv'] = True
        except NoSuchElementException:
            pass

        # 주차, 반려동물 동반, 노키즈존
        try:
            # 정보 탭으로 이동
            xpath = """//a[.//span[contains(text(),'정보')]]"""
            elements = driver.find_elements(By.XPATH, xpath)
            elem = elements[0]
            elem.send_keys(Keys.ENTER)
            time.sleep(0.5)

            # 주차
            try:
                xpath = "//div[h2[text()='주차']]"
                element = driver.find_element(By.XPATH, xpath)

                text = element.text

                if "주차불가" in text:
                    # print("무료 주차 여부: X")
                    store_dict["free_parking"] = False
                elif "주차가능" in text:
                    store_dict['free_parking'] = True                    
                if "발렛가능" in text:
                    store_dict['valet_parking'] = True
                else:
                    store_dict['valet_parking'] = False
            except:
                pass
            
        except Exception as e:
            print(e)
            pass

        print(store_dict)

    driver.switch_to.default_content()
    driver.find_element(By.XPATH, empty_root)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))
    driver.find_element(By.XPATH, empty_searchIframe)
    next_page_button = driver.find_element(By.XPATH, "//a[span[contains(text(),'다음페이지')]]")
    next_page_button.click()

===============1번째 매장===============
{'name': '에뚜왈 가로수길점', 'category': '베이커리', 'instagram_link': 'https://www.instagram.com/patisserie___etoile/?igshid=OGQ5ZDc2ODk2ZA%3D%3D', 'visitor_review_count': 1886, 'blog_review_count': 1609, 'distance_from_subway': 0, 'on_tv': False, 'is_parking': False, 'free_parking': False, 'valet_parking': False, 'no_kids': False, 'no_pet': True, 'seoul_michelin': False}
===============2번째 매장===============
인스타그램 없음
===============3번째 매장===============
list index out of range
{'name': '마님닭발 신사점', 'category': '닭발', 'instagram_link': 'https://www.instagram.com/manim_sinsa', 'visitor_review_count': 25, 'blog_review_count': 5, 'distance_from_subway': 0, 'on_tv': False, 'is_parking': False, 'free_parking': False, 'valet_parking': False, 'no_kids': False, 'no_pet': True, 'seoul_michelin': False}
===============4번째 매장===============


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app-root"]"}
  (Session info: chrome=120.0.6099.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104b684dc chromedriver + 4162780
1   chromedriver                        0x0000000104b60664 chromedriver + 4130404
2   chromedriver                        0x00000001047b7bc0 chromedriver + 293824
3   chromedriver                        0x00000001047fe040 chromedriver + 581696
4   chromedriver                        0x000000010483dab0 chromedriver + 842416
5   chromedriver                        0x00000001047f16bc chromedriver + 530108
6   chromedriver                        0x00000001047f2930 chromedriver + 534832
7   chromedriver                        0x0000000104b2de08 chromedriver + 3923464
8   chromedriver       

In [71]:
for i in range(1, 50):
    print("="*10 + f"{i}" + "="*10)
    driver = webdriver.Chrome()
    driver.get("https://map.naver.com/")

    wait = WebDriverWait(driver, 10)

    driver.implicitly_wait(5)
    driver.find_element(By.XPATH, empty_root)

    css_selector = ".input_search"
    elem = driver.find_element(By.CSS_SELECTOR, css_selector)

    time.sleep(1.5)

    elem.send_keys("신사역 맛집")
    time.sleep(1.5)

    elem.send_keys(Keys.RETURN)

    searchIframe = driver.find_element(By.XPATH, """//*[@id="searchIframe"]""")
        
    # Iframe 이동
    driver.switch_to.frame(searchIframe)
    # 네이버지도는 클릭이나 복사할 영역을 선택하지 않을 경우, 셀레니움의 크롤링 기능이 제대로 적용되지 않음
    driver.find_element(By.XPATH, empty_searchIframe)
                
    xpath = f"""//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]/a[1]"""
    elem = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
    print(elem)
    print(elem.text)
    print(elem.tag_name)

    time.sleep(2)
    driver.execute_script("arguments[0].click();", elem)
    # elem.click()
    # elem.send_keys(Keys.ENTER)

    # 가장 상위 frame으로 이동 후 entryIframe으로 이동
    driver.switch_to.default_content()
    driver.find_element(By.XPATH, empty_root)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))
    driver.find_element(By.XPATH, empty_entryIframe)

    # 매장 이름, 카테고리
    store_name_xpath = """//*[@id="_title"]/div/span"""
    elem = wait.until(EC.visibility_of_all_elements_located((By.XPATH, store_name_xpath)))
    print(elem[0].text)
    print(elem[1].text)

    time.sleep(2)

    driver.quit()

==========1==========
<selenium.webdriver.remote.webelement.WebElement (session="06c75dfb71668d09bc03a3e1beaa130c", element="CC0813FE69D0515A5401D06A03AC642A_element_1748")>
논논
네이버예약
쿠폰
이자카야
a
논논
이자카야
==========2==========
<selenium.webdriver.remote.webelement.WebElement (session="8b6f942d1040bb94950ee806c1a425d0", element="5B9D2881816AA2C02937992F5A359F33_element_1875")>
파스타 트리오
네이버페이
네이버예약
네이버톡톡
쿠폰
양식
a
파스타 트리오
양식
==========3==========
<selenium.webdriver.remote.webelement.WebElement (session="336025ff2de1e45108935029a56b69cc", element="E3C718CEDA984A4113682C3B394584A3_element_1890")>
신사화로구이
네이버톡톡
육류,고기요리
a
신사화로구이
육류,고기요리
==========4==========
<selenium.webdriver.remote.webelement.WebElement (session="5a8a58c34f94adb75e45bed340b3c49c", element="7CEDC592CEF38140965717403C2EF2E3_element_1740")>
압구정곱창 가로수길직영점곱창,막창,양
a
압구정곱창 가로수길직영점
곱창,막창,양
==========5==========
<selenium.webdriver.remote.webelement.WebElement (session="a092112bc1bace2ccf7cb34ca2b91d9a", element="29A0BAB91B8D46FC1E9A378A

In [140]:
driver = webdriver.Chrome()

driver.get("https://map.naver.com/")

wait = WebDriverWait(driver, 10)

css_selector = ".input_search"
elem = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector)))
# elem = driver.find_element(by="css selector", value=css_selector)

elem.send_keys("신사역 맛집")
elem.send_keys(Keys.RETURN)

# # 한 페이지의 모든 매장 loading
# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# driver.execute_script("window.scrollTo(0, 0);")

for i in range(1,51):
    
    print(f"========={i}번째 매장=========")

    # 가장 상위 frame으로 이동 후 searchIframe으로 이동
    driver.switch_to.default_content()
    wait = WebDriverWait(driver, 10)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))

    # i번째 매장에 대한 <a> 태그 찾아서 해상 위치까지 이동 후 click
    xpath = f"/html/body/div[3]/div/div[3]/div[1]/ul/li[{i}]/div[1]/a"
    elem = driver.find_element(By.XPATH, value=xpath)
    # 스크롤을 해당 요소 위치까지 이동 후 클릭
    driver.execute_script("arguments[0].scrollIntoView(true);", elem)
    elem.click()

    time.sleep(3)

    # 가장 상위 frame으로 이동 후 entryIframe으로 이동
    driver.switch_to.default_content()
    wait = WebDriverWait(driver, 10)
    # wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))
    entry_iframe = driver.find_element(By.ID, "entryIframe")
    driver.switch_to.frame(entry_iframe)


    time.sleep(2)

    # 매장 이름 get
    # store_name_xpath = "/html/body/div[3]/div/div/div/div[2]/div[1]/div[1]/div/span[1]"
    store_name_xpath = """//*[@id="_title"]/div/span[1]"""
    elem = wait.until(EC.visibility_of_element_located((By.XPATH, store_name_xpath)))
    # elem = driver.find_element(By.XPATH, value="/html/body/div[3]/div/div/div/div[2]/div[1]/div[1]/div/span[1]")
    store_name = elem.text

    # 매장 분류 get
    # store_category_xpath = "/html/body/div[3]/div/div/div/div[2]/div[1]/div[1]/div/span[2]"
    store_category_xpath = """//*[@id="_title"]/div/span[2]"""
    elem = wait.until(EC.visibility_of_element_located((By.XPATH, store_category_xpath)))
    # elem = driver.find_element(By.XPATH, value="/html/body/div[3]/div/div/div/div[2]/div[1]/div[1]/div/span[2]")
    store_category = elem.text

    print(f"매장 이름: {store_name}")
    print(f"매장 분류: {store_category}")
    
    # 인스타그램 계정 여부 확인 및 링크 get
    try:
        elem = driver.find_element(By.XPATH, value= "//a[contains(text(), '인스타그램')]")
        href = elem.get_attribute('href')

        print(f"인스타그램 계정: {href}")
    except:
        print("인스타그램 계정 없음")
        continue

    # 방문자 리뷰 개수
    try:
        elem = driver.find_element(By.XPATH, value="//a[contains(text(), '방문자리뷰')]")
        text = elem.text
        text = text.replace(",", "")

        visit_review_num = re.findall(r'\d+', text)[0]
        if visit_review_num:
            print(f"방문자 리뷰 {visit_review_num}건")
    except:
        print("방문자 리뷰 없음")

    # 블로그 리뷰 개수
    try:
        elem = driver.find_element(By.XPATH, value="//a[contains(text(), '블로그리뷰')]")
        text = elem.text
        text = text.replace(",", "")

        blog_review_number = re.findall(r'\d+', text)[0]
        if blog_review_number:
            print(f"블로그 리뷰 {blog_review_number}건")
    except:
        print("블로그 리뷰 없음")

    # 지하철역 출구로부터 거리
    try:
        elem = wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/div")))
        # elem = driver.find_element(By.XPATH, value="/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/div")
        text = elem.text

        numbers = re.findall(r'\d+', text)
        if numbers:
            distance_from_station_exit = numbers[-1]
            print(f"지하철역으로부터의 거리: {distance_from_station_exit}m")
    except NoSuchElementException:
        print("해당 XPath를 가진 div 태그가 없습니다.")

=========1번째 매장=========
매장 이름: 빠레뜨한남 가로수길점
매장 분류: 양식
인스타그램 계정 없음
=========2번째 매장=========
매장 이름: 초이다이닝 신사가로수길점
매장 분류: 일식당
인스타그램 계정: https://www.instagram.com/choi_dining/
방문자 리뷰 1935건
블로그 리뷰 657건
지하철역으로부터의 거리: 648m
=========3번째 매장=========
매장 이름: 봄의정원 가로수길점
매장 분류: 양식
인스타그램 계정: https://www.instagram.com/spring__garden_
방문자 리뷰 827건
블로그 리뷰 1839건
지하철역으로부터의 거리: 239m
=========4번째 매장=========
매장 이름: 메리고라운드 스테이크
매장 분류: 양식
인스타그램 계정: https://www.instagram.com/merry_go_round_steak
방문자 리뷰 3149건
블로그 리뷰 4845건
지하철역으로부터의 거리: 561m
=========5번째 매장=========
매장 이름: 포사로
매장 분류: 이자카야
인스타그램 계정: https://www.instagram.com/fosaro.izakaya?igshid=MzRlODBiNWFlZA==
방문자 리뷰 180건
블로그 리뷰 564건
지하철역으로부터의 거리: 295m
=========6번째 매장=========
매장 이름: 따우전드 신사점
매장 분류: 카페,디저트
인스타그램 계정: https://www.instagram.com/thousand_official_
방문자 리뷰 65건
블로그 리뷰 117건
지하철역으로부터의 거리: 578m
=========7번째 매장=========
매장 이름: 심퍼티쿠시 가로수길점
매장 분류: 양식
인스타그램 계정: https://www.instagram.com/szimpatikus.seoul
방문자 리뷰 552건
블로그 리뷰 1533건
지하철역으로부터의 거리: 184m
=========

In [148]:
driver = webdriver.Chrome()

driver.get("https://map.naver.com/")

wait = WebDriverWait(driver, 10)

css_selector = ".input_search"
elem = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector)))
# elem = driver.find_element(by="css selector", value=css_selector)

elem.send_keys("신사역 맛집")
elem.send_keys(Keys.RETURN)

for i in range(1,51):
    
    print(f"=============={i}번째 매장==============")

    # 가장 상위 frame으로 이동 후 searchIframe으로 이동
    driver.switch_to.default_content()
    wait = WebDriverWait(driver, 10)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))

    # i번째 매장에 대한 <a> 태그 찾아서 해상 위치까지 이동 후 click
    xpath = f"/html/body/div[3]/div/div[3]/div[1]/ul/li[{i}]/div[1]/a"
    
    # elem = driver.find_element(By.XPATH, value=xpath)
    elem = wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
    # 스크롤을 해당 요소 위치까지 이동 후 클릭
    if i!=1:
        driver.execute_script("arguments[0].scrollIntoView(true);", elem)
    elem.click()

    time.sleep(3)

    # 가장 상위 frame으로 이동 후 entryIframe으로 이동
    driver.switch_to.default_content()
    wait = WebDriverWait(driver, 10)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    time.sleep(2)

    # 매장 이름 get
    store_name_xpath = """//*[@id="_title"]/div/span[1]"""
    elem = wait.until(EC.visibility_of_element_located((By.XPATH, store_name_xpath)))
    store_name = elem.text

    # 매장 분류 get
    store_category_xpath = """//*[@id="_title"]/div/span[2]"""
    elem = wait.until(EC.visibility_of_element_located((By.XPATH, store_category_xpath)))
    store_category = elem.text

    print(f"매장 이름: {store_name}")
    print(f"매장 분류: {store_category}")
    
    # 인스타그램 계정 여부 확인 및 링크 get
    try:
        elem = driver.find_element(By.XPATH, value= "//a[contains(text(), '인스타그램')]")
        href = elem.get_attribute('href')

        print(f"인스타그램 계정: {href}")
    except:
        print("인스타그램 계정 없음")
        continue

    # 방문자 리뷰 개수
    try:
        elem = driver.find_element(By.XPATH, value="//a[contains(text(), '방문자리뷰')]")
        text = elem.text
        text = text.replace(",", "")

        visit_review_num = re.findall(r'\d+', text)[0]
        if visit_review_num:
            print(f"방문자 리뷰 {visit_review_num}건")
    except:
        print("방문자 리뷰 없음")

    # 블로그 리뷰 개수
    try:
        elem = driver.find_element(By.XPATH, value="//a[contains(text(), '블로그리뷰')]")
        text = elem.text
        text = text.replace(",", "")

        blog_review_number = re.findall(r'\d+', text)[0]
        if blog_review_number:
            print(f"블로그 리뷰 {blog_review_number}건")
    except:
        print("블로그 리뷰 없음")

    # 지하철역 출구로부터 거리
    try:
        elem = wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/div")))
        # elem = driver.find_element(By.XPATH, value="/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/div")
        text = elem.text

        numbers = re.findall(r'\d+', text)
        if numbers:
            distance_from_station_exit = numbers[-1]
            print(f"지하철역으로부터의 거리: {distance_from_station_exit}m")
    except NoSuchElementException:
        print("해당 XPath를 가진 div 태그가 없습니다.")

=========1번째 매장=========
우천식육식당 신사점
네이버예약
네이버톡톡
쿠폰
소고기구이
clicked!!
매장 이름: 우천식육식당 신사점
매장 분류: 소고기구이
인스타그램 계정: https://www.instagram.com/woo1000sinsa
방문자 리뷰 155건
블로그 리뷰 605건
지하철역으로부터의 거리: 206m
=========2번째 매장=========
오늘도한우 강남논현점
네이버예약
쿠폰
소고기구이
clicked!!
매장 이름: 오늘도한우 강남논현점
매장 분류: 소고기구이
인스타그램 계정: https://www.instagram.com/today_hanwoo_day
방문자 리뷰 144건
블로그 리뷰 125건
지하철역으로부터의 거리: 229m
=========3번째 매장=========
봄의정원 가로수길점
네이버예약
양식
clicked!!
매장 이름: 봄의정원 가로수길점
매장 분류: 양식
인스타그램 계정: https://www.instagram.com/spring__garden_
방문자 리뷰 829건
블로그 리뷰 1839건
지하철역으로부터의 거리: 239m
=========4번째 매장=========
메리고라운드 스테이크
네이버페이
네이버예약
네이버주문
네이버톡톡
양식
clicked!!
매장 이름: 메리고라운드 스테이크
매장 분류: 양식
인스타그램 계정: https://www.instagram.com/merry_go_round_steak
방문자 리뷰 3149건
블로그 리뷰 4845건
지하철역으로부터의 거리: 561m
=========5번째 매장=========
포사로
네이버페이
네이버예약
이자카야
clicked!!
매장 이름: 포사로
매장 분류: 이자카야
인스타그램 계정: https://www.instagram.com/fosaro.izakaya?igshid=MzRlODBiNWFlZA==
방문자 리뷰 180건
블로그 리뷰 564건
지하철역으로부터의 거리: 295m
=========6번째 매장=========
따우전드 신사점카페,디저트

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000104f404dc chromedriver + 4162780
1   chromedriver                        0x0000000104f38664 chromedriver + 4130404
2   chromedriver                        0x0000000104b8fbc0 chromedriver + 293824
3   chromedriver                        0x0000000104bd6040 chromedriver + 581696
4   chromedriver                        0x0000000104c15ab0 chromedriver + 842416
5   chromedriver                        0x0000000104bc96bc chromedriver + 530108
6   chromedriver                        0x0000000104bca930 chromedriver + 534832
7   chromedriver                        0x0000000104f05e08 chromedriver + 3923464
8   chromedriver                        0x0000000104f0a3dc chromedriver + 3941340
9   chromedriver                        0x0000000104eee038 chromedriver + 3825720
10  chromedriver                        0x0000000104f0af3c chromedriver + 3944252
11  chromedriver                        0x0000000104ee06f4 chro

In [154]:
driver = webdriver.Chrome()

driver.get("https://map.naver.com/")

wait = WebDriverWait(driver, 10)

css_selector = ".input_search"
elem = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector)))

elem.send_keys("신사역 맛집")
elem.send_keys(Keys.RETURN)

for i in range(1,51):
    
    print(f"=============={i}번째 매장==============")

    # 가장 상위 frame으로 이동 후 searchIframe으로 이동
    driver.switch_to.default_content()
    wait = WebDriverWait(driver, 10)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))

    # i번째 매장에 대한 <a> 태그 찾아서 해상 위치까지 이동 후 click
    xpath = f"/html/body/div[3]/div/div[3]/div[1]/ul/li[{i}]/div[1]/a"
    # elem = driver.find_element(By.XPATH, value=xpath)
    elem = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
    # 스크롤을 해당 요소 위치까지 이동 후 클릭
    if i!=1:
        driver.execute_script("arguments[0].scrollIntoView(true);", elem)
    elem.click()

    time.sleep(3)

    # 가장 상위 frame으로 이동 후 entryIframe으로 이동
    driver.switch_to.default_content()
    wait = WebDriverWait(driver, 10)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    time.sleep(2)

    # 매장 이름 get
    store_name_xpath = """//*[@id="_title"]/div/span[1]"""
    elem = wait.until(EC.visibility_of_element_located((By.XPATH, store_name_xpath)))
    store_name = elem.text

    # 매장 분류 get
    store_category_xpath = """//*[@id="_title"]/div/span[2]"""
    elem = wait.until(EC.visibility_of_element_located((By.XPATH, store_category_xpath)))
    store_category = elem.text

    print(f"매장 이름: {store_name}")
    print(f"매장 분류: {store_category}")
    
    # 인스타그램 계정 여부 확인 및 링크 get
    try:
        elem = driver.find_element(By.XPATH, value= "//a[contains(text(), '인스타그램')]")
        href = elem.get_attribute('href')

        print(f"인스타그램 계정: {href}")
    except:
        print("인스타그램 계정 없음")
        continue

    # 방문자 리뷰 개수
    try:
        elem = driver.find_element(By.XPATH, value="//a[contains(text(), '방문자리뷰')]")
        text = elem.text
        text = text.replace(",", "")

        visit_review_num = re.findall(r'\d+', text)[0]
        if visit_review_num:
            print(f"방문자 리뷰 {visit_review_num}건")
    except:
        print("방문자 리뷰 없음")

    # 블로그 리뷰 개수
    try:
        elem = driver.find_element(By.XPATH, value="//a[contains(text(), '블로그리뷰')]")
        text = elem.text
        text = text.replace(",", "")

        blog_review_number = re.findall(r'\d+', text)[0]
        if blog_review_number:
            print(f"블로그 리뷰 {blog_review_number}건")
    except:
        print("블로그 리뷰 없음")

    # 지하철역 출구로부터 거리
    try:
        elem = wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/div")))
        # elem = driver.find_element(By.XPATH, value="/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/div")
        text = elem.text

        numbers = re.findall(r'\d+', text)
        if numbers:
            distance_from_station_exit = numbers[-1]
            print(f"지하철역으로부터의 거리: {distance_from_station_exit}m")
    except NoSuchElementException:
        print("해당 XPath를 가진 div 태그가 없습니다.")

==============1번째 매장==============


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001032244dc chromedriver + 4162780
1   chromedriver                        0x000000010321c664 chromedriver + 4130404
2   chromedriver                        0x0000000102e73bc0 chromedriver + 293824
3   chromedriver                        0x0000000102eba040 chromedriver + 581696
4   chromedriver                        0x0000000102ef9ab0 chromedriver + 842416
5   chromedriver                        0x0000000102ead6bc chromedriver + 530108
6   chromedriver                        0x0000000102eae930 chromedriver + 534832
7   chromedriver                        0x00000001031e9e08 chromedriver + 3923464
8   chromedriver                        0x00000001031ee3dc chromedriver + 3941340
9   chromedriver                        0x00000001031d2038 chromedriver + 3825720
10  chromedriver                        0x00000001031eef3c chromedriver + 3944252
11  chromedriver                        0x00000001031c46f4 chro